# Midterm Project - Apriori Algorithm

**Author** : *Joshua Kobuskie*

## Part 1 Details
1. Create 10 (or any number of, not less than 5) items usually seen in
Amazon, K-mart, or any other supermarkets (e.g. diapers, clothes, etc.).
2. Create a database of at least 20 transactions each containing some of these
items. Save the transaction in a CSV file.
3. Repeat (1) by creating 4 additional, different databases each containing at
least 20 transactions.
4. Note: You can create these transactions and datasets manually, download
them from the net, or use the examples I will provide. In any case, add a
note to your report where and how you built your data sets.
5. The items and transaction must not be random so that your code is
deterministic

In [ ]:
# Create Amazon transactions an dataset from provided example
import csv

amazonData  = [ ["Transaction ID", "Transaction"],
["Trans1", "A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch"],
"Trans2", "A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies"
"Trans3", "A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition"
"Trans4", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition", "Beginning Programming with Java"
"Trans5", "Android Programming: The Big Nerd Ranch", "Beginning Programming with Java", "Java 8 Pocket Guide"
"Trans6", "A Beginner’s Guide", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition"
"Trans7", "A Beginner’s Guide", "Head First Java 2nd Edition", "Beginning Programming with Java"
"Trans8", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch"
"Trans9", "Java For Dummies", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition", "Beginning Programming with Java"
"Trans10", "Beginning Programming with Java", "Java 8 Pocket Guide", "C++ Programming in Easy Steps"
"Trans11", "A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch"
"Trans12", "A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "HTML and CSS: Design and Build Websites"
"Trans13", "A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Java 8 Pocket Guide", "HTML and CSS: Design and Build Websites"
"Trans14", "Java For Dummies", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition"
"Trans15", "Java For Dummies", "Android Programming: The Big Nerd Ranch"
"Trans16", "A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch"
"Trans17", "A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch"
"Trans18", "Head First Java 2nd Edition", "Beginning Programming with Java", "Java 8 Pocket Guide"
"Trans19", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition"
"Trans20", "A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies"]

with open("amazonTransactions.csv", "w", newline="") as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(amazonData)

## Part 2 Details
* Implement the brute force method to generate frequent items and generate
association rules.
* The brute force method for finding frequent itemsets works as follows.
Enumerate and generate all possible 1-itemsets and 2-itemsets. There are
30 items, so there are 435 possible 2-itemsets totally. Check to see whether
each possible 1-itemset/2-itemset is frequent. Then enumerate and generate
all possible 3-itemsets. There are 4060 possible 3-itemsets totally. Check to
see whether each possible 3-itemset is :frequent. Keep on doing so until
you see none of the possible k-itemsets is :frequent for some k, at which
point the brute force method terminates without generating (k+ 1 )-
itemsets.

## Part 3 Details
Use an existing Apriori implementation from Python libraries/packages to
verify the results from your brute force algorithm implementation.
* Use Python existing package for fpgrowth (as known as fp-tree algorithm)
to generate the items and rules.
* Compare the results from your brute-force, Apriori, and FP-Tree/Growth.
* Do the three algorithms produce the same results?
* Report the timing performance for all three algorithms as well.
* Which one is faster?
So, for all three algorithms, generate and print out all the association rules and
the input transactions for each of the 5 transactional databases you created/used.
The data set selection, support, and confidence must be user-specified
parameters, so the output should show different rules with respect to different
databases and different support/confidence.
Make sure to show multiple support and confidence results for each data set.
You should prompt the user only once for the input and reuse for the three
algorithms in each run.
The items and transactions must be clear and easy to identify. Your
program should show the performance time for each algorithm.

## Part 4 Details
Github & Jupyter Notebook.
* After you finish your code in development and testing
and make sure it works, and prepare the report (meaning
all heavy lifting job is done ), Create a Github
repository in https://github.com/. Your account must be
with your NJIT email not your personal email (unless if
you have to, but indicate that in your report as well).
* Load your project to the repository.
* Create Jupyter notebook for your work to show the
output, for more info visit https://jupyter.org/
* Give me ya54@njit.edu access as a collaborator to your
repository. (If we have a grader, you give him/her access
too).
* Add Github link to your repository to your report.